## Install libraries

In [1]:
!pip install openai
!pip install langchain
!pip install langchain_core
!pip install langchain_openai
!pip install lancedb
!pip install ipywidgets
!pip install tantivy
!pip install markdown
!pip install bs4
!pip install inflect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.

## Cleanup

In [2]:
!rm -rf ./real-estate-listings.json
!rm -rf ./real-estate-embeddings-db

## Imports

In [20]:
from google.colab import userdata
import pandas as pd
import numpy as np
import openai
import os
import json
import re

from langchain.llms import OpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import LanceDB
from lancedb.rerankers import LinearCombinationReranker

import lancedb
from lancedb.embeddings import get_registry
from lancedb.pydantic import LanceModel, Vector

from bs4 import BeautifulSoup
from markdown import markdown

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider, FloatSlider

import inflect

## Keys and Constants

In [4]:
DATA_FILE = "homematch.json"
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')


## Generate and save (or load) listings using LangChain

In [5]:
system_prompt = """
You are an expert real estate agent in New York City in the USA.
"""

human_prompt= """
      Generate at least 15 real estate listings. Use your imagination to generate listings.
      Be sure to include real listings as well.
      Distribute listings across the 5 NYC boroughs.
      The listings must be in the JSON array of dictionaries with each item in the format as shown below:

        {
          "location": "Upper West Side",
          "list_price": 1000000,
          "bedrooms": 3,
          "bathrooms": 2,
          "square_feet": 1000,
          "monthly_hoa": 1000,
          "school_rating": 4.5,
          "description": "A beautiful pre-war building completely restored to modern living. Modern kitchen, new appliances, interior decorated by a famous architect. Ready to move in. Internet connection to Verizon FIOS and Comcast. Satellite TV dish can be installed on the roof. A serene neighborhood with access to shopping, dining, and entertainment. A real gem for starting families. Great schools. Close to subway."
        }
    """

if os.path.isfile(path=DATA_FILE):
  f = open(DATA_FILE, "r")
  real_estate_listings = f.read()
  f.close()
else:
  chat = ChatOpenAI(temperature=1)
  messages = [
      SystemMessage(
          content=system_prompt
      ),
      HumanMessage(
          content=human_prompt
      ),
  ]
  aimessage = chat.invoke(messages)
  real_estate_listings = json.loads(aimessage.json())["content"]

md_text = markdown(real_estate_listings)
real_estate_listings_text = ''.join(BeautifulSoup(md_text).findAll(string=True))
real_estate_listings_json = real_estate_listings_text.replace('json\n', '')
print(real_estate_listings_json)
f = open(DATA_FILE, "w")
f.write(real_estate_listings_json)
f.close()


[
  {
    "location": "Upper West Side",
    "list_price": 2100000,
    "bedrooms": 4,
    "bathrooms": 3.5,
    "square_feet": 2000,
    "monthly_hoa": 1200,
    "school_rating": 4.8,
    "description": "Luxurious apartment with panoramic views of Central Park. High-end finishes, gourmet kitchen, spacious bedrooms, and a private terrace. Building amenities include a fitness center, concierge service, and rooftop garden. Close to upscale shopping and dining."
  },
  {
    "location": "Park Slope",
    "list_price": 1500000,
    "bedrooms": 3,
    "bathrooms": 2,
    "square_feet": 1500,
    "monthly_hoa": 800,
    "school_rating": 4.7,
    "description": "Charming brownstone in a prime Brooklyn location. Renovated kitchen, original details, hardwood floors, and a backyard garden. Walkable to Prospect Park, trendy cafes, and boutique shops. Easy access to public transportation."
  },
  {
    "location": "Astoria",
    "list_price": 800000,
    "bedrooms": 2,
    "bathrooms": 1,
    "squ

## Convert listings to embeddings and save in LanceDb

In [11]:
df = pd.read_json(DATA_FILE)
db = lancedb.connect("real-estate-embeddings-db")
embeddings = get_registry().get("openai").create()
class RealEstateListings(LanceModel):
    location: str
    list_price: float
    bedrooms: float
    bathrooms: float
    square_feet: float
    monthly_hoa: float
    school_rating: float
    description: str = embeddings.SourceField()
    description_vector: Vector(embeddings.ndims()) = embeddings.VectorField()

if 'listings' in db.table_names():
  table = db.open_table("listings")
else:
  table = db.create_table("listings", schema=RealEstateListings)
  data = df.apply(
      lambda row: {
          "location": row["location"],
          "list_price": row["list_price"],
          "bedrooms": row["bedrooms"],
          "bathrooms": row["bathrooms"],
          "square_feet": row["square_feet"],
          "monthly_hoa": row["monthly_hoa"],
          "school_rating": row["school_rating"],
          "description": row["description"],
      },
      axis=1,
  ).values.tolist()
  table.add(data)
print(table.to_pandas().head())

          location  list_price  bedrooms  bathrooms  square_feet  monthly_hoa  \
0  Upper West Side   2100000.0       4.0        3.5       2000.0       1200.0   
1       Park Slope   1500000.0       3.0        2.0       1500.0        800.0   
2          Astoria    800000.0       2.0        1.0       1000.0        500.0   
3           Harlem    650000.0       2.0        1.5       1200.0        400.0   
4     Forest Hills   1200000.0       3.0        2.5       1800.0        700.0   

   school_rating                                        description  \
0            4.8  Luxurious apartment with panoramic views of Ce...   
1            4.7  Charming brownstone in a prime Brooklyn locati...   
2            4.2  Modern condo with skyline views, open layout, ...   
3            3.9  Historic brownstone with character, exposed br...   
4            4.6  Spacious townhouse with a private backyard, up...   

                                  description_vector  
0  [0.005693865, 0.0062472997, 

## Inline User Interface to capture Buyer preferences

In [15]:
form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

form_items = [
    Box([Label(value='Max Price'), FloatSlider(min=1000000, max=5000000, step=10000, value=5000000)], layout=form_item_layout),
    Box([Label(value='Bed Rooms minimum'), FloatSlider(min=1, max=10, step=1)], layout=form_item_layout),
    Box([Label(value='Bath Rooms minimum'), FloatSlider(min=1, max=10, step=1)], layout=form_item_layout),
    Box([Label(value='School Ratings'), FloatSlider(min=1, max=5, step=1)], layout=form_item_layout),
    Box([Label(value='Square Footage'), FloatSlider(min=1000, max=5000, step=500)], layout=form_item_layout),
    Box([Label(value='Preferences'),
         Textarea(value="brownstone prewar historic")], layout=form_item_layout)
]

form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='50%'
))
form


Box(children=(Box(children=(Label(value='Max Price'), FloatSlider(value=5000000.0, max=5000000.0, min=1000000.…

In [16]:
max_price = form_items[0].children[1].value
bedrooms = form_items[1].children[1].value
bathrooms = form_items[2].children[1].value
school_rating = form_items[3].children[1].value
square_feet = form_items[4].children[1].value
preferences = form_items[5].children[1].value
print(max_price)
print(bedrooms)
print(bathrooms)
print(school_rating)
print(square_feet)
print(preferences)

5000000.0
1.0
1.0
1.0
1000.0
brownstone prewar historic


## Prefilter based on numeric preferences and vector search on textual preferences.

In [17]:
db = lancedb.connect("real-estate-embeddings-db")
table = db.open_table("listings")
print(table.to_pandas().head())
filterExpr = f"list_price < {max_price} and bedrooms > {bedrooms} and bathrooms > {bathrooms} and school_rating > {school_rating} and square_feet > {square_feet}"
filteredDf = table.search(preferences, vector_column_name="description_vector").where(filterExpr, prefilter=True).limit(5).to_pandas()
filteredDf.head()

          location  list_price  bedrooms  bathrooms  square_feet  monthly_hoa  \
0  Upper West Side   2100000.0       4.0        3.5       2000.0       1200.0   
1       Park Slope   1500000.0       3.0        2.0       1500.0        800.0   
2          Astoria    800000.0       2.0        1.0       1000.0        500.0   
3           Harlem    650000.0       2.0        1.5       1200.0        400.0   
4     Forest Hills   1200000.0       3.0        2.5       1800.0        700.0   

   school_rating                                        description  \
0            4.8  Luxurious apartment with panoramic views of Ce...   
1            4.7  Charming brownstone in a prime Brooklyn locati...   
2            4.2  Modern condo with skyline views, open layout, ...   
3            3.9  Historic brownstone with character, exposed br...   
4            4.6  Spacious townhouse with a private backyard, up...   

                                  description_vector  
0  [0.005693865, 0.0062472997, 

,location,list_price,bedrooms,bathrooms,square_feet,monthly_hoa,school_rating,description,description_vector,_distance
0,Harlem,650000.0,2.0,1.5,1200.0,400.0,3.9,"Historic brownstone with character, exposed br...","[-0.011094097, 0.0060560717, 0.0075896676, -0....",0.288407
1,Park Slope,1500000.0,3.0,2.0,1500.0,800.0,4.7,Charming brownstone in a prime Brooklyn locati...,"[0.007816437, -0.0036009385, 0.013955637, -0.0...",0.335021
2,Bay Ridge,780000.0,4.0,2.0,1400.0,450.0,4.3,"Classic brick row house with modern upgrades, ...","[-0.002175228, 0.018290205, -0.020733254, -0.0...",0.411511
3,Staten Island,620000.0,3.0,2.0,1600.0,300.0,4.0,Recently renovated single-family home with an ...,"[0.0017241267, 0.02850898, 0.0054127323, 0.004...",0.422422
4,Morningside Heights,950000.0,2.0,1.5,1100.0,600.0,4.1,"Renovated co-op with skyline views, granite co...","[0.0084790485, 3.3990877e-05, 0.022458019, -0....",0.423045


## Personalize Listings

In [21]:
def generate_output(query, df):
    context = ""
    p = inflect.engine()
    for index, row in df.iterrows():
      context += "Located in " + row["location"] + " with a list price of " + p.number_to_words(int(row["list_price"])) + " that has " + p.number_to_words(int(row["bedrooms"])) + " bedrooms, " + p.number_to_words(int(row["bathrooms"])) + " bathrooms, " + str(int(row["square_feet"])) + " square feet, and a school rating of " + str(row["school_rating"]) + ". " + row["description"] + ". " + "\n\n"
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "You are an expert real estate agent that answers user's questions based on the context provided.\nDo not make up an answer if you do not know it, stay within the bounds of the context provided, if you don't know the answer, say that you don't have enough information on the topic!",
            },
            {"role": "user", "content": f"CONTEXT: {context}\nQUERY: {query}"},
            {"role": "user", "content": "ANSWER:"},
        ],
    )

    response = response.choices[0].message.content.strip()
    return response

In [22]:
generate_output("Give me a factual summary of top 3 listings", filteredDf)

'Based on the information provided, here is a factual summary of the top 3 listings:\n\n1. Located in Park Slope with a list price of one million, five hundred thousand, this property offers three bedrooms, two bathrooms, and spans 1500 square feet. The school rating is 4.7, indicating a high-quality education environment. The charming brownstone features a renovated kitchen, original details, hardwood floors, and a backyard garden. The location is prime in Brooklyn, walkable to Prospect Park, trendy cafes, and boutique shops with easy access to public transportation.\n\n2. Located in Morningside Heights with a list price of nine hundred and fifty thousand, this property features two bedrooms, one bathroom, and 1100 square feet of space. The school rating is 4.1, suggesting a good educational setting. The renovated co-op offers skyline views, granite countertops, stainless steel appliances, and hardwood floors. Building amenities include a doorman, laundry room, and bike storage. It is

In [23]:
generate_output("Recommend a listing that is close to subways with top school rating", filteredDf)

'I recommend the property located in Park Slope with a list price of one million, five hundred thousand. It has a school rating of 4.7, which is the highest among the options provided. Additionally, this property is in a prime Brooklyn location, close to public transportation, including subways, making it convenient for commuting.'

In [24]:
generate_output("Recommend at least two listings that are close to subways with top school rating", filteredDf)

'Based on the information provided, the listings in Harlem and Morningside Heights would be the best recommendations for you as they are close to subways and have a top school rating.\n\n1. The listing in Harlem with a list price of six hundred and fifty thousand is close to transportation, which likely includes subway access, and has a school rating of 3.9.\n\n2. The listing in Morningside Heights priced at nine hundred and fifty thousand is steps away from Columbia University, parks, and transportation, making it easily accessible to subways. It also boasts a school rating of 4.1.\n\nBoth these listings offer proximity to subways and top school ratings, making them ideal choices for your criteria.'